In [ ]:
# 1. INSTALL DEPENDENCIES
!pip install -q sentence-transformers faiss-cpu datasets langchain==0.1.0 langchain-community==0.0.13 langchain-text-splitters

print("Installation complete!\n")

# 2. IMPORTS
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
import json
from tqdm import tqdm

# 3. LOAD DATA FROM HUGGING FACE
print("=" * 30)
print("LOADING DATASETS FROM HUGGING FACE")

# Dataset 1: OPP-115
print("\nDownloading Dataset 1: OPP-115 (alzoubi36/opp_115)...")
dataset_1 = load_dataset("alzoubi36/opp_115")

# Dataset 2: PrivacyPolicy
print("\nDownloading Dataset 2: PrivacyPolicy (sjsq/PrivacyPolicy)...")
dataset_2 = load_dataset("sjsq/PrivacyPolicy")


# Dataset 3: Kaggle ToSDR
print("\nLoading Dataset 3: Kaggle ToSDR (sonu1607/tosdr-terms-of-service-corpus)...")
# Since we don't have kaggle CLI, we check if file exists locally
import os
import pandas as pd

kaggle_files = [f for f in os.listdir('.') if f.endswith('.csv') or f.endswith('.json')]
dataset_3 = None

# Try to find the file
target_file = None
for f in kaggle_files:
    if 'tosdr' in f.lower() or 'corpus' in f.lower():
        target_file = f
        break

if target_file:
    print(f"   Found local file: {target_file}")
    try:
        if target_file.endswith('.csv'):
            dataset_3 = pd.read_csv(target_file)
        else:
            dataset_3 = pd.read_json(target_file)
        print(f"   - Kaggle ToSDR: {len(dataset_3)} examples")
    except Exception as e:
        print(f"   ❌ Error loading file: {e}")
else:
    print("   ⚠️  Kaggle dataset file not found locally.")
    print("   Please download the dataset from https://www.kaggle.com/datasets/sonu1607/tosdr-terms-of-service-corpus")
    print("   and place it in the same directory as this notebook (e.g., 'tosdr_corpus.csv').")
print(f"\nDatasets loaded successfully!")
print(f"   - OPP-115 Train: {len(dataset_1['train'])} examples")
print(f"   - PrivacyPolicy Train: {len(dataset_2['train'])} examples")

# 4. CONVERT TO LANGCHAIN DOCUMENTS 
print("\n" + "=" * 30)
print("PREPARING DOCUMENTS")

documents = []

# Process Dataset 1 (OPP-115)
print("\nProcessing OPP-115 dataset...")
for split in ['train', 'validation', 'test']:
    if split in dataset_1:
        print(f"  - Processing {split} split...")
        for i, item in enumerate(tqdm(dataset_1[split])):
            doc = Document(
                page_content=item['text'],
                metadata={
                    'source': 'opp_115',
                    'split': split,
                    'index': i,
                    'original_id': f"opp_{split}_{i}"
                }
            )
            documents.append(doc)

# Process Dataset 2 (sjsq/PrivacyPolicy)
print("\nProcessing sjsq/PrivacyPolicy dataset...")
for split in dataset_2.keys():
    print(f"  - Processing {split} split...")
    for i, item in enumerate(tqdm(dataset_2[split])):
        # Normalize column names (this dataset uses 'Text')
        text_content = item.get('Text', item.get('text', ''))
        if text_content and text_content != 'Null':
             doc = Document(
                page_content=text_content,
                metadata={
                    'source': 'sjsq_privacy_policy',
                    'split': split,
                    'index': i,
                    'filename': item.get('filename', 'unknown')
                }
            )
             documents.append(doc)


# Process Dataset 3 (Kaggle ToSDR)
if dataset_3 is not None:
    print("\nProcessing Kaggle ToSDR dataset...")
    # Inspect columns to find text column
    text_col = None
    for col in dataset_3.columns:
        if 'text' in col.lower() or 'content' in col.lower() or 'terms' in col.lower() or 'service' in col.lower():
            text_col = col
            break
    
    if text_col:
        print(f"   Using column '{text_col}' for text content")
        for i, row in tqdm(dataset_3.iterrows(), total=len(dataset_3)):
            text_content = row[text_col]
            if isinstance(text_content, str) and len(text_content) > 100:
                doc = Document(
                    page_content=text_content,
                    metadata={
                        'source': 'kaggle_tosdr',
                        'index': i,
                        'filename': target_file
                    }
                )
                documents.append(doc)
    else:
        print(f"   ⚠️  Could not find a suitable text column in {dataset_3.columns}")
print(f"\nTotal documents prepared: {len(documents)}")
# 5. CHUNK THE DOCUMENTS
print("\n" + "=" * 30)
print("CHUNKING DOCUMENTS FOR RETRIEVAL")

print("\nSplitting text into searchable chunks...")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    separators=["\n\n", "\n", ". ", " "],
    length_function=len,
)

chunks = text_splitter.split_documents(documents)

print(f"\n✅ Created {len(chunks)} searchable chunks")
print(f"   Average chunk size: ~{sum(len(c.page_content) for c in chunks) // len(chunks)} characters")

# 6. CREATE EMBEDDINGS MODEL
print("\n" + "=" * 30)
print("LOADING EMBEDDING MODEL FROM HUGGING FACE")

print("\nLoading: sentence-transformers/all-MiniLM-L6-v2")
print("   - Model size: 80MB")
print("   - Embedding dimensions: 384")
print("   - License: Apache 2.0 (free for all uses)")

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

print("\nEmbedding model loaded successfully!")

# 7. BUILD VECTOR DATABASE
print("\n" + "=" * 30)
print("BUILDING FAISS VECTOR DATABASE")

print("\n🔨 Creating embeddings for all chunks...")
print("   (This may take 2-3 minutes for 3,000+ chunks)")

vectorstore = FAISS.from_documents(chunks, embeddings)

print("\nVector database created successfully!")
print(f"   Total vectors: {vectorstore.index.ntotal}")

# 8. TEST THE DATABASE
print("\n" + "=" * 30)
print("TESTING RETRIEVAL")

test_queries = [
    "Can the company sell my personal data to third parties?",
    "Do they use my content to train AI models?",
    "Can I delete all my data from their servers?",
    "Am I forced into arbitration instead of court?"
]

print("\nRunning test queries...\n")

for query in test_queries:
    print(f"Query: '{query}'")
    results = vectorstore.similarity_search(query, k=2)
    print(f"Found {len(results)} relevant chunks")
    print(f"   Top result preview: {results[0].page_content[:150]}...")
    print()

# 9. SAVE THE DATABASE
print("=" * 30)
print("SAVING DATABASE")

print("\n💾 Saving FAISS database to disk...")
vectorstore.save_local("faiss_index_tos_hf")

print("Database saved to: faiss_index_tos_hf/")

# 10. CREATE METADATA FILE
metadata = {
    "dataset": "alzoubi36/opp_115",
    "total_documents": len(documents),
    "total_chunks": len(chunks),
    "embedding_model": "sentence-transformers/all-MiniLM-L6-v2",
    "embedding_dimension": 384,
    "chunk_size": 800,
    "chunk_overlap": 100,
    "vector_database": "FAISS",
    "date_created": "2024-12",
    "license": "Apache 2.0",
    "use_case": "Terms of Service Analysis - IAT 360"
}

with open('faiss_index_tos_hf/metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("Metadata saved")

# 11. ZIP FOR DOWNLOAD
print("\nCreating zip file for download...")
!zip -r -q faiss_index_tos_hf.zip faiss_index_tos_hf

print("\n" + "=" * 30)
print("SUCCESS! DATABASE READY FOR DEPLOYMENT")

# Display file sizes
print("\n📊 File sizes:")
!du -sh faiss_index_tos_hf
!du -sh faiss_index_tos_hf.zip

📦 Installing packages...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.9 MB/s eta 0:00:00
ERROR: Cannot install langchain-community==0.0.13, langchain-text-splitters==0.2.0, langchain-text-splitters==0.2.1, langchain-text-splitters==0.2.2, langchain-text-splitters==0.2.4, langchain-text-splitters==0.3.0, langchain-text-splitters==0.3.1, langchain-text-splitters==0.3.10, langchain-text-splitters==0.3.11, langchain-text-splitters==0.3.2, langchain-text-splitters==0.3.3, langchain-text-splitters==0.3.4, langchain-text-splitters==0.3.5, langchain-text-splitters==0.3.6, langchain-text-splitters==0.3.7, langchain-text-splitters==0.3.8, langchain-text-splitters==0.3.9, langchain-text-splitters==1.0.0 and langchain==0.1.0 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts
✅ Installation complete!

STEP 1: LOADING DATASET FROM HUGGI

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



✅ Dataset loaded successfully!
   - Train set: 2185 examples
   - Validation set: 550 examples
   - Test set: 697 examples

📋 Sample document:
------------------------------------------------------------
Text preview:  ""Contact Us"" Link If you contact us through the ""Contact Us"" link on this site, we ask you for information such as your first name, e-mail address, and other information, so we can respond to your questions and comments. You may choose to provide additional information as well. ...

Available fields: ['text', 'label']
------------------------------------------------------------

STEP 2: PREPARING DOCUMENTS

Processing train set...


100%|██████████| 2185/2185 [00:00<00:00, 34835.62it/s]



Processing validation set...


...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


✅ Embedding model loaded successfully!

STEP 5: BUILDING FAISS VECTOR DATABASE

🔨 Creating embeddings for all chunks...
   (This may take 2-3 minutes for 3,000+ chunks)

✅ Vector database created successfully!
   Total vectors: 3929

STEP 6: TESTING RETRIEVAL

🧪 Running test queries...

Query: 'Can the company sell my personal data to third parties?'
✅ Found 2 relevant chunks
   Top result preview: Note, that we will not share your Personally Identifiable Information with third parties for their marketing purposes without obtaining your prior con...

Query: 'Do they use my content to train AI models?'
✅ Found 2 relevant chunks
   Top result preview: Our automated systems analyze your content (including emails) to provide you personally relevant product features, such as customized search results, ...

Query: 'Can I delete all my data from their servers?'
✅ Found 2 relevant chunks
   Top result preview: Remember that even after you cancel your account, copies of some information from y